<a href="https://colab.research.google.com/github/CIS-522/course-content/blob/W3_homework_updates/tutorials/W3_MLPs/student/W3_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS-522 Week 3 Homework


**Instructor:** Konrad Kording

**Content Creators:** Arash Ash, Jordan Lei

In [ ]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = 'value' #@param {type:"string"}
my_pod = 'euclidean-wombat' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion','astute-jellyfish', 'quantum-herring']

# start timing
import time
try:t0;
except NameError: t0 = time.time()


---
# Setup

In [ ]:
# imports
import os
import csv
import json
import random
import pathlib

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from IPython.display import HTML, display

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dev, torch.get_num_threads()

In [ ]:
# @title Install Kaggle API
# %%capture
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.10
!kaggle -v

In [ ]:
# @title Helper functions
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis(False)
    plt.show()

def progress(epoch, loss, epochs=100):
    return HTML("""
        <label for="file">Training loss: {loss}</label>
        <progress
            value='{epoch}'
            max='{epochs}',
            style='width: 100%'
        >
            {epoch}
        </progress>
    """.format(loss=loss, epoch=epoch, epochs=epochs))

## Part 1. Kaggle Competition
To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (`https://www.kaggle.com/<username>/account`) and select 'Create API Token'. This will trigger the download of `kaggle.json`, a file containing your API credentials. Place this file in Colab Files on the left menu.

Then join the competition and accept the rules at https://www.kaggle.com/c/permuted-animal-faces/rules.
Now we can handle the submisions here using the Kaggle API:

In [ ]:
# reegistering your API key
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c permuted-animal-faces
!unzip permuted-animal-faces.zip

Now it's your turn to train a competitive model using what you learned about MLPs and show it off to the rest of the world in the Kaggle leaderboard.

Here we provided you the same code as in the tutorial and the submission code, which you can run and get a baseline result. But we encourage you to get creative and combine your previous ML know-how since nothing is off the table in this competition.

In [ ]:
class Net(nn.Module):
    def __init__(self, actv, num_inputs, hidden_units, num_outputs):
        super(Net, self).__init__()

        exec('self.actv = nn.%s'%actv)   # [TO-DO]

        self.layers = nn.ModuleList()
        for i in range(len(hidden_units)):
          next_num_inputs = hidden_units[i] 
          self.layers += [nn.Linear(num_inputs, next_num_inputs)]   # [TO-DO]
          num_inputs = next_num_inputs

        self.out = nn.Linear(num_inputs, num_outputs)

    def forward(self, x):
        # flattening
        x = x.view(x.shape[0], -1)   # [TO-DO]

        for layer in self.layers:
          x = self.actv(layer(x))  # [TO-DO]
        x = self.out(x) # [TO-DO]
        return x

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
data = np.array(df)
X_train = torch.tensor(data[:, 1:]).float()/255
y_train = torch.tensor(data[:, 0]).long()

In [ ]:
batch_size = 128
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, drop_last=True,
                        shuffle=True, num_workers=10)

In [ ]:
def train_test_classification(net, criterion, optimizer,
                              train_loader,
                              num_epochs=1, verbose=True,
                              training_plot=True):
  if verbose:
    progress_bar = display(progress(0, 0, num_epochs), display_id=True)

  net.train()
  training_losses = []
  for epoch in range(num_epochs):  # loop over the dataset multiple times
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs = inputs.to(dev).float()
          labels = labels.to(dev).long()

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)

          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          if verbose:
            training_losses += [loss.item()]
            running_loss += loss.item()
            if i % 10 == 9:    # update every 10 mini-batches
                progress_bar.update(progress(epoch+1, running_loss / 10, num_epochs))
                running_loss = 0.0

  net.eval()
  def test(data_loader):
    correct = 0
    total = 0
    for data in data_loader:
        inputs, labels = data
        inputs = inputs.to(dev).float()
        labels = labels.to(dev).long()

        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    return total, acc

  train_total, train_acc = test(train_loader)

  if verbose:
    print('Accuracy on the %d training samples: %0.2f %%' % (train_total, train_acc))

  if training_plot:
    plt.plot(training_losses)
    plt.xlabel('Batch')
    plt.ylabel('Training loss')
    plt.show()
  
  return train_acc

In [ ]:
net = Net('ReLU()', 3*32*32, [128, 32], 3).to(dev) 
criterion = nn.MultiMarginLoss(margin=1.0)
optimizer = optim.Adam(net.parameters(), lr=3e-4)
train_acc = train_test_classification(net, criterion, optimizer,
                                      train_loader,
                                      num_epochs=50)

In [ ]:
# Predict test set
df = pd.read_csv('test.csv')
data = np.array(df)
X_test = torch.tensor(data).float()/255

net.eval()
y_pred = net(X_test.to(dev))
labels_pred = torch.argmax(y_pred, dim=1)

In [ ]:
# create the submission file
header = ['ImageId', 'Label']
with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    
    for i in range(len(labels_pred)):
      writer.writerow([i+1] + [labels_pred[i].item()])

In [ ]:
# submit
!kaggle competitions submit permuted-animal-faces -f submission.csv -m "Result with same model as in the tutorial"

Note that you can submit maximum 20 times a day. Participants will need to wait until the next UTC day after submitting the maximum number of daily submissions.

And finally you can hand select 2 submissions among your previous ones to be used for the final ranking.

# Part 2. Deep Learning, Deeper Thinking



In [6]:
#@markdown **Time Check (just run this cell)**
try: t1;
except NameError: t1 = time.time()

## 2.1 Our Data-Processing Wish List

By now you should be pretty familiar with classifying animal faces using multi-layer perceptrons. MLPs make very few assumptions about the underlying structure of the data, which make them very flexible in approximating a wide variety of functions. However, sometimes it can be helpful to constrain the set of possible functions we want to approximate by introducing some "rules" that these functions need to follow. We call these rules **inductive biases**. 

For example, one inductive bias might be the rule that "A cat is a cat, no matter where it is placed in the image. An upper-left-corner cat is just as much of a cat as a lower-right-corner cat." That describes an inductive bias called **translational invariance**, which basically says that the class of an object is invariant to translation (it doesn't matter where it is, as long as it's there). If we could have a model that preserves translational invariance, chances are it would be helpful in finding us good models for image processing (more on this later). 

**Now that you're familiar with the concept of inductive biases, please do the following:**
* List one inductive bias other than translational invariance that would be good to have for image processing
* Suppose you're trying to develop a model to assign credit scores to people based on their personal data. What kind of inductive biases or data guarantees would you want to impose on this system?

In [2]:
#Delete the starter text, enter your response, and run the cell.
q_2_1 = '''
Your Response to Q 2.1 Here

'''

### 2.2.1 **America Under Watch**
Sometimes lack of good data isn't the problem. What happens if we collect data *everywhere*? In many countries around the world, facial recognition is used by law enforcement to conduct wide-scale law enforcement. Thankfully that's not the case in the United States. Right? 

*... Right?* 

Read the following:

* https://www.americaunderwatch.com/

**In 200-300 words, please answer the following questions.** 
* In America Under Watch, the Georgetown Law Center proposes a moratorium (temporary freeze) on the police use of facial recognition. Do you agree or disagree with this choice, and why? 
* The article mentions the fact that *facial surveillance is disproportionately likely to impact communities of color and communities with lower socio-economic status (SES)*. Think about what this means about what amount + what kinds of data law enforcement collect on people of color. How might this perpetuate cycles of incarceration for people of color in the U.S.?
* If facial recognition were to be put in place in a highly secure location, like the White House, what kind of inductive biases would you want to ensure that such a system would have? What guarantees might you want to have about the data that is used to train the network, the data that is collected, and what is done with the data afterward? 

In [3]:
#Delete the starter text, enter your response, and run the cell.
q_2_2_1 = '''
Your Response to Q 2.2.1 Here (200 - 300 words)
'''

In [7]:
#@markdown **Time Check (just run this cell)**
try: t2;
except NameError: t2 = time.time()

### 2.2.2 **Grab Some Popcorn**


In [ ]:
#@markdown That's enough reading for today, don't you think? Grab some popcorn, run this cell, and watch this video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="jZjmlJPJgug", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

**In 200-300 words, please answer the following questions**
* In the segment, John Oliver mentions that Clearview AI uses "publicly available data", which includes public profile information on Facebook, LinkedIn, Twitter, and Instagram. Clearview argues that this information has all been willingly made public. Do you agree? Why or why not?
* Some prominent data scientists claim that "data/algorithms don't have bias, people have bias". Do you agree with this sentiment? Why or why not?


In [4]:
#Delete the starter text, enter your response, and run the cell.
q_2_2_2 = '''
Your Response to Q 2.2.2 Here (100 - 200 words)

'''

# Submit to Airtable
**Don't forget to contribute to the conversation in your pod slack channel.** You can do so by copying and pasting some or all of the answers to the above questions, or by commenting and responding to other people's posts. *italicized text*


In [9]:
#@markdown #Run Cell to Show Airtable Form
#@markdown ##**Confirm your answers and then click "Submit"**
import numpy as np
import time
from IPython.display import IFrame
def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefills = "&".join(["prefill_%s=%s"%(key, fields[key]) for key in fields])
  src = src + prefills
  src = "+".join(src.split(" "))
  return src

#autofill time if it is not present
try: t0;
except NameError: t0 = time.time()
try: t1;
except NameError: t1 = time.time()
try: t2;
except NameError: t2 = time.time()
try: t3;
except NameError: t3 = time.time()

#autofill fields if they are not present
#a missing pennkey and pod will result in an Airtable warning
#which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""
try: my_pod;
except NameError: my_pod = "Select"
try: q_2_1;
except NameError: q_2_1= ""
try: q_2_2_1;
except NameError: q_2_2_1 = ""
try: q_2_2_2;
except NameError: q_2_2_2 = ""


times = np.array([t1,t2,t3])-t0

fields = {"pennkey": my_pennkey,
          "pod": my_pod,
          "q_2_1": q_2_1,
          "q_2_2_1": q_2_2_1,
          "q_2_2_2": q_2_2_2,
          "cumulative_times": times}

src = "https://airtable.com/embed/shrzuKoahe4ZQKvND?"

#now instead of the original source url, we do: src = prefill_form(src, fields)
display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))